In [ ]:
#imports
from google.colab import drive
drive.mount('/content/drive')

import re
from transformers import BertTokenizer, BertModel
import torch

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
left_wing_path = '/content/drive/My Drive/sparta lab/transcripts/left_videos.txt'
print(left_wing_path)

/content/drive/My Drive/sparta lab/transcripts/left_videos.txt


In [40]:
# load BERT large cased model from HuggingFace
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertModel.from_pretrained('bert-large-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Ignore this block below

In [27]:
# testing model
input_text = "The quick brown fox jumps over the lazy dog."
inputs = tokenizer(input_text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state
print(last_hidden_states[0][0])


tensor([-0.3982, -0.5164,  0.1184,  ..., -0.5752, -0.4897,  0.2587])


Setting left wing text variable

In [28]:
with open(left_wing_path, 'r', encoding='utf-8') as file:
    left_wing_text = file.read()
    # print(left_wing_text[:200])
portion_length = len(left_wing_text) // 400

# Extract the 1/20th portion of the text
smaller_text_portion = left_wing_text[:portion_length]
# print(smaller_text_portion)

Making segments for BERT

In [29]:
def segment_and_tokenize(text):
    # Split the text into segments
    sentences = re.split(r'(?<=[.!?])\s+', text)
    segments = []
    current_segment = ""

    for sentence in sentences:
        if len(tokenizer(current_segment + sentence)['input_ids']) > 512:
            segments.append(current_segment)
            current_segment = sentence
        else:
            current_segment += sentence

    segments.append(current_segment)  # Add the last segment
    # print(segments)

    # Tokenize segments
    tokenized_segments = [tokenizer(segment, truncation=True, padding='max_length', max_length=512) for segment in segments]

    return tokenized_segments

# Example usage
tokenized_segments = segment_and_tokenize(smaller_text_portion)


In [36]:
class LeftWingDataset(torch.utils.data.Dataset):
    def __init__(self, encodings_list):
        # Convert list of dicts to dict of lists
        self.encodings = {key: [dic[key] for dic in encodings_list] for key in encodings_list[0]}

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

dataset = LeftWingDataset(tokenized_segments)


In [32]:
# Load the BERT model
model = BertModel.from_pretrained('bert-large-uncased')


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [41]:
def extract_features(dataset):
    model.eval()  # Put the model in evaluation mode
    features = []

    with torch.no_grad():
        for item in dataset:
            # Ensure input has batch dimension
            input_ids = item['input_ids'].unsqueeze(0)
            attention_mask = item['attention_mask'].unsqueeze(0)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            features.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())

    return features

features = extract_features(dataset)
